# 层和块

我们先回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
'''
输入 X:        [2个样本, 20个特征]
    ↓
Linear(20→256)  [2个样本, 256个特征]
    ↓
ReLU()          [2个样本, 256个特征] (非线性)
    ↓
Linear(256→10)  [2个样本, 10个特征] ← 最终输出
'''
# 网络结构：20 → 256 → 10，是一个简单的多层感知机。
net = nn.Sequential(nn.Linear(20, 256), # 第一个全连接层，输入20个特征，输出256个特征
                    nn.ReLU(), # ReLU激活函数（非线性变换）
                    nn.Linear(256, 10) # 第二个全连接层，输入256个特征，输出10个特征
                    )

X = torch.rand(2, 20) # 生成一个随机输入张量，形状为 (2, 20)
# 将输入数据 X 传入网络，依次经过三层计算
net(X)

tensor([[ 0.1023,  0.0488,  0.0394, -0.0471,  0.2309,  0.3416,  0.1725, -0.1227,
          0.1627, -0.0382],
        [ 0.0837, -0.0374,  0.1049,  0.0411,  0.2304,  0.4862,  0.2797, -0.1901,
          0.3293,  0.0701]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
'''
输入 X  (2, 20)
    ↓
hidden layer (Linear 20→256)
    ↓
ReLU激活
    ↓
out layer (Linear 256→10)
    ↓
输出 (2, 10)
'''
class MLP(nn.Module): # 继承自PyTorch的nn.Module
    # 构造函数，初始化模型时自动调用。
    def __init__(self):
        super().__init__() # 必须调用，用于初始化父类nn.Module的内部结构
        self.hidden = nn.Linear(20, 256) # 隐藏层，全连接层，输入20维 → 输出256维。
        self.out = nn.Linear(256, 10) # 输出层，全连接层，输入256维 → 输出10维。

    def forward(self, X):
        '''
        self.hidden(X)：输入 X 经过隐藏层
        F.relu(...)：对隐藏层输出应用ReLU激活函数
        self.out(...)：ReLU的结果再经过输出层，得到最终输出
        '''
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0015,  0.1076, -0.0305,  0.0246, -0.0502,  0.0464, -0.1783,  0.0289,
          0.0258,  0.1449],
        [ 0.0976,  0.1328, -0.0528,  0.1033, -0.0841,  0.0831, -0.3594,  0.0434,
          0.1123,  0.2958]], grad_fn=<AddmmBackward0>)

顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args): # *args：接收所有传入的层（如Linear,ReLU）
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module # str(idx)：用字符串索引（如"0", "1"）作为键，这是PyTorch的要求

    def forward(self, X):
        # X = layer3(layer2(layer1(X)))
        for block in self._modules.values():
            X = block(X)
        return X
# 实例化自定义模型，结构为Linear→ReLU→Linear
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2080,  0.0358, -0.0123, -0.0585,  0.0683, -0.2248, -0.1701, -0.0357,
          0.1599, -0.1337],
        [-0.1628,  0.0929, -0.0391,  0.0271,  0.1110, -0.2769, -0.0741,  0.0316,
          0.1797, -0.1436]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 创建固定权重矩阵（20×20随机矩阵，requires_grad=False表示不参与梯度计算，不会被训练更新）
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        # 可训练的线性层（输入输出均为20维）
        self.linear = nn.Linear(20, 20) 

    def forward(self, X):
        X = self.linear(X) # 第一次：线性变换1
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 固定权重乘法+偏置+ReLU
        X = self.linear(X) # 第二次：重复使用同一个线性层（权重共享）
        while X.abs().sum() > 1: # 动态控制流：当绝对值之和大于1时循环
            X /= 2 # 逐元素减半
        return X.sum() # 返回所有元素之和（标量输出）

net = FixedHiddenMLP()
net(X)

tensor(-0.0652, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 结构：net子网络（20→64→32）+linear层（32→16）
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    # 前向传播
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), # 20 → 16
                        nn.Linear(16, 20), # 16 → 20（维度回升）
                        FixedHiddenMLP()) # 20 → 标量（调用之前的特殊模型）

chimera(X)

tensor(-0.3501, grad_fn=<SumBackward0>)